In [1]:
import base64
import requests
import os
import autogen
import pprint


In [2]:
print(dir(autogen))
print(autogen.__file__)


['Agent', 'AssistantAgent', 'ChatCompletion', 'Completion', 'ConversableAgent', 'DEFAULT_MODEL', 'FAST_MODEL', 'GroupChat', 'GroupChatManager', 'UserProxyAgent', '__builtins__', '__cached__', '__doc__', '__file__', '__loader__', '__name__', '__package__', '__path__', '__spec__', '__version__', 'agentchat', 'code_utils', 'config_list_from_dotenv', 'config_list_from_json', 'config_list_from_models', 'config_list_gpt4_gpt35', 'config_list_openai_aoai', 'get_config_list', 'logger', 'logging', 'oai', 'version']
/Users/serkanhaytac/experimental-projects/autogen-habittracker/venv/lib/python3.11/site-packages/autogen/__init__.py


In [3]:
config_list_gpt4 = autogen.config_list_from_json(
    "OAI_CONFIG_LIST",
    filter_dict={
        "model": ["gpt-4", "gpt-4-0314", "gpt4", "gpt-4-32k", "gpt-4-32k-0314", "gpt-4-32k-v0314"],
    },
)
print(config_list_gpt4)


[]


In [4]:
# OpenAI API Key
api_key = os.getenv("OPENAI_API_KEY")


In [5]:
# Function to encode the image
def encode_image(image_path):
  with open(image_path, "rb") as image_file:
    return base64.b64encode(image_file.read()).decode('utf-8')

# Path to your image
def encode_image_to_base64(image_path):
    # This function reads the image file and converts it to a base64 string
    with open(image_path, "rb") as image_file:
        encoded_string = base64.b64encode(image_file.read()).decode()
    return encoded_string

def send_image_to_api(image_path):
    # Encode the image to base64
    base64_image = encode_image_to_base64(image_path)
  

    # Prepare headers
    headers = {
      "Content-Type": "application/json",
      "Authorization": f"Bearer {api_key}"
    }

    # Prepare payload
    payload = {
      "model": "gpt-4-vision-preview",
      "messages": [
        {
          "role": "user",
          "content": [
            {
              "type": "image_url",
              "image_url": {
                "url": f"data:image/jpeg;base64,{base64_image}"
              }
            }
          ]
        }
      ],
      "max_tokens": 1000
    }

    # Send the request
    response = requests.post("https://api.openai.com/v1/chat/completions", headers=headers, json=payload)
    return pprint.pprint(response.json())


In [9]:


llm_config = {
    "config_list": config_list_gpt4,
    "seed": 4,
    "temperature": 0,
    "functions": [
        {
            "name": "send_image_to_api",
            "description": "Gets image path and sends it to the API.",
            "parameters": {
                "type": "object",
                "properties": {
                    "image_path": {
                        "type": "string",
                        "description": "Valid path.",
                    }
                },
                "required": ["image_path"],
            },
        }
    ]
}
coder = autogen.AssistantAgent(
    name="Assistant",  # the default assistant agent is capable of solving problems with code
    system_message="""You are a react native developer and you are working on creating a new react native app """,
    llm_config=llm_config,
)
user_proxy = autogen.UserProxyAgent(
   name="user_proxy",
   is_termination_msg=lambda x: x.get("content", "") and x.get("content", "").rstrip().endswith("TERMINATE"),
   human_input_mode="NEVER",
   system_message="""Reply TERMINATE if the task has been solved at full satisfaction.
   Otherwise, reply CONTINUE, or the why the task is not solved yet""",
   code_execution_config={"last_n_messages": 3, "work_dir": "groupchat"},
   llm_config=llm_config,
)

user_proxy.register_function(function_map={
    "send_image_to_api": send_image_to_api
})


In [16]:
# user_proxy.initiate_chat(
#     coder,
#     message="""Upload /Users/serkanhaytac/experimental-projects/autogen-habittracker/app_screenshot.jpg
#     image using send_image_to_api registered function. Once image is uploaded, come up with a plan to create a habit tracker app.
#     App should be react native app that has the ability to keep track of habits"
#     """
# )
groupchat = autogen.GroupChat(agents=[user_proxy, coder], messages=[], max_round=12)
manager = autogen.GroupChatManager(groupchat=groupchat, llm_config=llm_config)
user_proxy.initiate_chat(manager, message="""Upload /Users/serkanhaytac/experimental-projects/autogen-habittracker/app_screenshot.jpg
image using send_image_to_api registered function. Once image is uploaded, come up with a plan to create a habit tracker app.
App should be react native app that has the ability to keep track of habits. Start building the app""")


user_proxy (to chat_manager):

Upload /Users/serkanhaytac/experimental-projects/autogen-habittracker/app_screenshot.jpg
image using send_image_to_api registered function. Once image is uploaded, come up with a plan to create a habit tracker app.
App should be react native app that has the ability to keep track of habits. Start building the app

--------------------------------------------------------------------------------


GroupChat is underpopulated with 2 agents. Direct communication would be more efficient.


[autogen.oai.completion: 11-18 13:13:02] {788} WARNING - Completion was provided with a config_list, but the list was empty. Adopting default OpenAI behavior, which reads from the 'model' parameter instead.
[autogen.oai.completion: 11-18 13:13:02] {788} WARNING - Completion was provided with a config_list, but the list was empty. Adopting default OpenAI behavior, which reads from the 'model' parameter instead.
Assistant (to chat_manager):

***** Suggested function Call: send_image_to_api *****
Arguments: 
{
  "image_path": "/Users/serkanhaytac/experimental-projects/autogen-habittracker/app_screenshot.jpg"
}
******************************************************

--------------------------------------------------------------------------------

>>>>>>>> EXECUTING FUNCTION send_image_to_api...
{'choices': [{'finish_details': {'stop': '<|fim_suffix|>', 'type': 'stop'},
              'index': 0,
              'message': {'content': 'This appears to be a screenshot of a '
                   

GroupChat is underpopulated with 2 agents. Direct communication would be more efficient.


[autogen.oai.completion: 11-18 13:13:09] {788} WARNING - Completion was provided with a config_list, but the list was empty. Adopting default OpenAI behavior, which reads from the 'model' parameter instead.
[autogen.oai.completion: 11-18 13:13:10] {788} WARNING - Completion was provided with a config_list, but the list was empty. Adopting default OpenAI behavior, which reads from the 'model' parameter instead.
Assistant (to chat_manager):

Great, the image has been uploaded successfully. Now, let's plan for the habit tracker app.

The app will have the following features:

1. **User Authentication**: Users will be able to register and login to their accounts. This will allow them to have a personalized experience and keep track of their habits.

2. **Habit Creation**: Users will be able to create new habits. They will provide the name of the habit, the frequency (daily, weekly, etc.), and optionally a reminder time.

3. **Habit Tracking**: Users will be able to mark their habits as don

...

GroupChat is underpopulated with 2 agents. Direct communication would be more efficient.


[autogen.oai.completion: 11-18 13:14:58] {788} WARNING - Completion was provided with a config_list, but the list was empty. Adopting default OpenAI behavior, which reads from the 'model' parameter instead.

>>>>>>>> EXECUTING CODE BLOCK 0 (inferred language is bash)...


execute_code was called without specifying a value for use_docker. Since the python docker package is not available, code will be run natively. Note: this fallback behavior is subject to change



>>>>>>>> EXECUTING CODE BLOCK 1 (inferred language is bash)...


execute_code was called without specifying a value for use_docker. Since the python docker package is not available, code will be run natively. Note: this fallback behavior is subject to change
